<a href="https://colab.research.google.com/github/jpordoy/RSNA-2024-Lumbar-Spine-Degenerative-Classification/blob/main/RSNA_2024_Lumbar_Spine_Degenerative_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!pip install pydicom

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 28.0 MB/s eta 0:00:00


In [1]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'rsna-2024-lumbar-spine-degenerative-classification:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F71549%2F8561470%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240713%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240713T154049Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D16456db4721f570942963a87a3092d4a28bc37fd425f94d167b47db4c424a73785e36e403d4225b8f632b9890e0b26e5aa9b44597ada4f70df06d6ed12870c765d1bab8e39571f3755c81533a451e9f5b0e478ccaef9db158e80af3840bdc25c70e4a414c03f10306309dca915dc1cafe2748cd0f2eee8d538aae0be1e45a96c599857c161562b59901ae009b454a5afb90c10e0da1c63e07c899a36e8fc6e6ec856b93002cb3b7839feff7e9b5450123286a2fb1d84c0190cfb0e7b5b61bf980ba971eb2cd18de891b040785bf12f91e8baf66745d8370edaf514f8354addabbf15525ceafc773f290d07bcfc66b2fd0ec2d6a3f71434daf49e42f102d426df'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')

[==================================================] 30303207710 bytes downloaded
Downloaded and uncompressed: rsna-2024-lumbar-spine-degenerative-classification
Data source import complete.


In [46]:
import pandas as pd
import numpy as np
import pydicom
from PIL import Image as PILImage
import os
import cv2
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.model_selection import train_test_split
import os
import glob
import pandas as pd

# Load the CSV files
df_train = pd.read_csv("/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train.csv")
df_train_desc = pd.read_csv("/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train_label_coordinates.csv")

# Add the image path to the descriptive DataFrame
df_train_desc['image_path'] = "/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train_images/" + df_train_desc['study_id'].astype(str) + "/" + df_train_desc['series_id'].astype(str) + "/" + df_train_desc['instance_number'].astype(str) + ".dcm"

# Reshape the training DataFrame
df_train_melted = df_train.melt(id_vars=['study_id'], var_name='condition_level', value_name='severity')

# Extract 'condition' and 'level' from 'condition_level'
df_train_melted[['conditions', 'level']] = df_train_melted['condition_level'].str.rsplit('_', n=2, expand=True).iloc[:, 1:]
df_train_melted['condition'] = df_train_melted['condition_level'].apply(lambda x: '_'.join(x.split('_')[:-2])).str.replace("_"," ").str.title()
df_train_melted['level'] = df_train_melted['conditions'].str.upper() + "/" + df_train_melted['level'].str.upper()

# Drop unnecessary columns
df_train_melted = df_train_melted.drop(columns=['condition_level', 'conditions'])

# Merge the DataFrames on 'study_id', 'level', and 'condition'
df_final = pd.merge(df_train_desc, df_train_melted, on=['study_id', 'level', 'condition'], how='inner')

# Display a sample of the final DataFrame
df_final.sample(5)

,study_id,series_id,instance_number,condition,level,x,y,image_path,severity
13339,1190473557,230363452,17,Right Neural Foraminal Narrowing,L2/L3,243.548906,220.254925,/kaggle/input/rsna-2024-lumbar-spine-degenerat...,Normal/Mild
3015,264945797,3695845416,9,Left Subarticular Stenosis,L2/L3,140.558491,118.656604,/kaggle/input/rsna-2024-lumbar-spine-degenerat...,Normal/Mild
29289,2607462358,384543934,9,Spinal Canal Stenosis,L4/L5,293.782114,350.074797,/kaggle/input/rsna-2024-lumbar-spine-degenerat...,Normal/Mild
5697,492697281,1082839900,13,Left Neural Foraminal Narrowing,L4/L5,248.892456,294.325524,/kaggle/input/rsna-2024-lumbar-spine-degenerat...,Normal/Mild
29937,2661101390,163444571,16,Left Neural Foraminal Narrowing,L5/S1,156.578411,257.494908,/kaggle/input/rsna-2024-lumbar-spine-degenerat...,Normal/Mild


In [47]:
from sklearn.model_selection import train_test_split

# Split df_final into training and validation sets
df_train_labeled, df_val_labeled = train_test_split(df_final, test_size=0.2, random_state=333)

print(f"Total labeled training images: {len(df_train_labeled)}")
print(f"Total labeled validation images: {len(df_val_labeled)}")

Total labeled training images: 38953
Total labeled validation images: 9739


In [48]:
labels_list = list(set(df_train_labeled['severity'].values))
label2id, id2label = dict(), dict()
for i, label in enumerate(labels_list):
    label2id[label] = i
    id2label[i] = label

print(id2label, '\n\n', label2id)

{0: 'Normal/Mild', 1: nan, 2: 'Severe', 3: 'Moderate'} 

 {'Normal/Mild': 0, nan: 1, 'Severe': 2, 'Moderate': 3}


In [55]:
import numpy as np
import pydicom
from PIL import Image as PILImage
from tensorflow.keras.utils import Sequence

class DataGenerator(Sequence):
    def __init__(self, image_paths, labels=None, batch_size=32, target_shape=(128, 128), shuffle=True):
        self.image_paths = image_paths
        self.labels = labels
        self.batch_size = batch_size
        self.target_shape = target_shape
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        return int(np.floor(len(self.image_paths) / self.batch_size))

    def __getitem__(self, index):
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        image_paths_temp = [self.image_paths[k] for k in indexes]
        if self.labels is not None:
            labels_temp = [self.labels[k] for k in indexes]
        else:
            labels_temp = None
        X, y = self.__data_generation(image_paths_temp, labels_temp)
        return X, y

    def on_epoch_end(self):
        self.indexes = np.arange(len(self.image_paths))
        if self.shuffle:
            np.random.shuffle(self.indexes)

    def __data_generation(self, image_paths_temp, labels_temp):
        X = np.empty((self.batch_size, *self.target_shape, 3))  # Ensure all images have 3 channels
        if labels_temp is not None:
            y = np.empty((self.batch_size), dtype=int)
        else:
            y = None
        for i, img_path in enumerate(image_paths_temp):
            img = pydicom.dcmread(img_path).pixel_array
            img = (img / np.max(img) * 255).astype(np.uint8)
            img = PILImage.fromarray(img).resize(self.target_shape)
            img = np.array(img)
            if img.ndim == 2:  # Convert grayscale to RGB
                img = np.stack((img,)*3, axis=-1)
            img = img / 255.0
            X[i,] = img
            if labels_temp is not None:
                y[i] = labels_temp[i]
        if labels_temp is not None:
            return X, y
        else:
            return X, None

# Prepare the data
train_data = {
    'image_path': df_train_labeled['image_path'].values,
    'label': df_train_labeled['severity'].map(label2id).values
}

val_data = {
    'image_path': df_val_labeled['image_path'].values,
    'label': df_val_labeled['severity'].map(label2id).values
}

# Define the generators
train_generator_labeled = DataGenerator(train_data['image_path'], train_data['label'])
val_generator = DataGenerator(val_data['image_path'], val_data['label'])





In [51]:
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Define the model using EfficientNetB0
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(128, 128, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
predictions = Dense(len(label2id), activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

# Define learning rate
learning_rate = 0.001  # Adjust the learning rate as needed

# Compile the model with Adam optimizer and custom learning rate
optimizer = Adam(learning_rate=learning_rate)
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Define batch size
batch_size = 128  # Adjust based on your system's capacity and performance

# Train the model
model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=5,
    batch_size=batch_size
)

Epoch 1/5
1217/1217 [==============================] - 560s 434ms/step - loss: 0.5561 - accuracy: 0.7856 - val_loss: 0.7209 - val_accuracy: 0.7734
Epoch 2/5
1217/1217 [==============================] - 526s 432ms/step - loss: 0.4960 - accuracy: 0.8016 - val_loss: 0.7406 - val_accuracy: 0.7733
Epoch 3/5
1217/1217 [==============================] - 528s 434ms/step - loss: 0.4759 - accuracy: 0.8061 - val_loss: 0.7743 - val_accuracy: 0.7735
Epoch 4/5
1217/1217 [==============================] - 527s 433ms/step - loss: 0.4583 - accuracy: 0.8099 - val_loss: 0.6356 - val_accuracy: 0.7746
Epoch 5/5
1217/1217 [==============================] - 525s 431ms/step - loss: 0.4410 - accuracy: 0.8135 - val_loss: 0.7303 - val_accuracy: 0.7772


In [52]:
from sklearn.metrics import classification_report, confusion_matrix

# Calculate total number of validation samples
total_val_samples = len(val_generator.labels)

# Generate predictions for the validation set
val_predictions = model.predict(val_generator, steps=len(val_generator), verbose=1)
val_predictions = np.argmax(val_predictions, axis=1)

# Ensure the length of predictions matches the total number of samples
if len(val_predictions) != total_val_samples:
    # Adjusting for potential batch size differences
    if len(val_predictions) < total_val_samples:
        # If predictions are shorter, it could be due to the last batch having fewer samples
        total_val_samples = len(val_predictions)
    else:
        # If predictions are longer, there might be an issue with how batches are processed
        raise ValueError(f"Number of predictions ({len(val_predictions)}) does not match number of validation samples ({total_val_samples})")

# Ensure target_names are strings
target_names = list(map(str, label2id.keys()))

# Print classification report and confusion matrix
print(classification_report(val_generator.labels[:total_val_samples], val_predictions, target_names=target_names))
print(confusion_matrix(val_generator.labels[:total_val_samples], val_predictions))

304/304 [==============================] - 107s 347ms/step
              precision    recall  f1-score   support

 Normal/Mild       0.77      0.99      0.87      7525
         nan       0.00      0.00      0.00         6
      Severe       0.00      0.00      0.00       590
    Moderate       0.22      0.01      0.03      1607

    accuracy                           0.77      9728
   macro avg       0.25      0.25      0.22      9728
weighted avg       0.63      0.77      0.68      9728

[[7434    0   16   75]
 [   6    0    0    0]
 [ 586    0    0    4]
 [1581    0    4   22]]


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [62]:
# Define SSL data generators
train_generator_labeled = DataGenerator(train_data['image_path'], train_data['label'])
train_generator_unlabeled = DataGenerator(unannotated_image_paths)

In [94]:
# Define SSL model architecture for three-class classification
def create_ssl_model(input_shape=(128, 128, 3), num_classes=3):
    # Shared convolutional base
    base_model = tf.keras.Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten()
    ])

    # Labeled data input
    input_labeled = Input(shape=input_shape, name='input_labeled')
    x_labeled = base_model(input_labeled)
    output_labeled = Dense(num_classes, activation='softmax', name='output_labeled')(x_labeled)

    # Unlabeled data input
    input_unlabeled = Input(shape=input_shape, name='input_unlabeled')
    x_unlabeled = base_model(input_unlabeled)
    output_unlabeled = Dense(3, activation='softmax', name='output_unlabeled')(x_unlabeled)

    # SSL model
    ssl_model = Model(inputs=[input_labeled, input_unlabeled], outputs=[output_labeled, output_unlabeled])

    return ssl_model

# Define SSL model
ssl_model = create_ssl_model(num_classes=3)

# Compile SSL model
ssl_model.compile(optimizer='adam',
                  loss={'output_labeled': 'sparse_categorical_crossentropy', 'output_unlabeled': 'categorical_crossentropy'},
                  metrics=['accuracy'])

# Compile SSL model
ssl_model.compile(optimizer='adam',
                  loss={'output_labeled': 'sparse_categorical_crossentropy', 'output_unlabeled': 'categorical_crossentropy'},
                  metrics=['accuracy'])

NameError: name 'num_classes' is not defined

In [97]:
import tensorflow as tf
import numpy as np

# Assuming you have already defined train_generator_labeled and train_generator_unlabeled

# Debug snippet to print shapes of inputs and targets
for (batch_labeled, _), (batch_unlabeled, _) in zip(train_generator_labeled, train_generator_unlabeled):
    print("Shapes - Labeled Data:")
    print(f"  batch_labeled.shape: {batch_labeled.shape}")

    if batch_unlabeled is not None:
        print("Shapes - Unlabeled Data:")
        print(f"  batch_unlabeled.shape: {batch_unlabeled.shape}")

    # Prepare targets for labeled and unlabeled data (just an example, adjust as needed)
    targets_labeled = {'output_labeled': batch_labeled, 'output_unlabeled': tf.one_hot(tf.zeros(tf.shape(batch_labeled)[0], dtype=tf.int32), depth=3)}
    if batch_unlabeled is not None:
        targets_unlabeled = {'output_labeled': batch_labeled, 'output_unlabeled': tf.one_hot(tf.zeros(tf.shape(batch_unlabeled)[0], dtype=tf.int32), depth=3)}
    else:
        targets_unlabeled = None

    if targets_labeled is not None and targets_unlabeled is not None:
        # Print targets shapes
        print("Shapes - Targets:")
        print(f"  targets_labeled['output_labeled'].shape: {targets_labeled['output_labeled'].shape}")
        print(f"  targets_labeled['output_unlabeled'].shape: {targets_labeled['output_unlabeled'].shape if targets_labeled['output_unlabeled'] is not None else 'N/A'}")
        if targets_unlabeled is not None:
            print(f"  targets_unlabeled['output_labeled'].shape: {targets_unlabeled['output_labeled'].shape}")
            print(f"  targets_unlabeled['output_unlabeled'].shape: {targets_unlabeled['output_unlabeled'].shape if targets_unlabeled['output_unlabeled'] is not None else 'N/A'}")
        else:
            print("  targets_unlabeled is None")

        # Break after printing the first batch for quick inspection
        break
    else:
        print("Targets are None. Skipping debug print.")
import tensorflow as tf
import numpy as np


# Now you can proceed to your training loop
num_epochs_ssl = 10

for epoch in range(num_epochs_ssl):
    print(f"Epoch {epoch+1}/{num_epochs_ssl}")

    for (batch_labeled, _), (batch_unlabeled, _) in zip(train_generator_labeled, train_generator_unlabeled):
        # Prepare inputs for labeled data
        inputs = {'input_labeled': batch_labeled}

        # Prepare targets for labeled data
        targets_labeled = {'output_labeled': batch_labeled}

        if batch_unlabeled is not None:
            # Prepare inputs for unlabeled data
            inputs['input_unlabeled'] = batch_unlabeled

            # Prepare targets for unlabeled data
            targets_unlabeled = {'output_labeled': batch_labeled}

            # Train the SSL model on both labeled and unlabeled data
            labeled_loss = ssl_model.train_on_batch(inputs, {'output_labeled': targets_labeled['output_labeled'],
                                                             'output_unlabeled': targets_unlabeled['output_labeled']})
        else:
            # Train the SSL model only on labeled data
            labeled_loss = ssl_model.train_on_batch(inputs, {'output_labeled': targets_labeled['output_labeled']})
            print("Skipping training on unlabeled data as batch_unlabeled is None.")

        # Optional: Log or print metrics for monitoring
        print(f"  Batch - Labeled Loss: {labeled_loss}")

print("Training completed.")

Shapes - Labeled Data:
  batch_labeled.shape: (32, 128, 128, 3)
Shapes - Unlabeled Data:
  batch_unlabeled.shape: (32, 128, 128, 3)
Shapes - Targets:
  targets_labeled['output_labeled'].shape: (32, 128, 128, 3)
  targets_labeled['output_unlabeled'].shape: (32, 3)
  targets_unlabeled['output_labeled'].shape: (32, 128, 128, 3)
  targets_unlabeled['output_unlabeled'].shape: (32, 3)
Epoch 1/10


TypeError: argument of type 'NoneType' is not iterable